In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
import spacy
from transformers import CLIPModel, CLIPProcessor
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
import re
import pandas as pd
import numpy as np

In [4]:
# Load CLIP Model and Processor
def load_clip_model(device):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    model.to(device)
    return model, processor

# Compute CLIP Embedding for Image
def get_image_embedding(model, processor, image, device):
    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_embedding = model.get_image_features(**inputs)
    return image_embedding.cpu().numpy()

# Compute CLIP Embedding for Text
def get_text_embedding(model, processor, text, device):
    inputs = processor(text=text, return_tensors="pt").to(device)
    with torch.no_grad():
        text_embedding = model.get_text_features(**inputs)
    return text_embedding.cpu().numpy()

# Calculate Cosine Similarity
def calculate_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2)[0][0]

# Extract adjectives from a given caption
spacy.require_gpu()
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

def extract_adjectives(caption):
    doc = nlp(caption)
    return [token.text for token in doc if token.pos_ == "ADJ"]

# Compare Adjective-Modified Captions
def compare_adjective_embeddings(caption, adjectives, image_embedding, model, processor, device):
    """
    Compare angular distance (angle) of image embedding with each adjective vs. adjective-free text embeddings.
    """
    results = []
    for adjective in adjectives:

        # Remove the selected adjective from the caption
        pattern = r'\b' + re.escape(adjective) + r'\b'
        adjective_free_caption = re.sub(pattern, '', caption).strip()

        # Get embeddings for the original and adjective-free captions
        original_embedding = get_text_embedding(model, processor, caption, device)
        adjective_free_embedding = get_text_embedding(model, processor, adjective_free_caption, device)

        # Calculate cosine similarities
        sim_with_adjective = calculate_similarity(image_embedding, original_embedding)
        sim_without_adjective = calculate_similarity(image_embedding, adjective_free_embedding)

        # Convert cosine similarity to angular distance (in radians)
        angle_with_adjective = np.arccos(sim_with_adjective)
        angle_without_adjective = np.arccos(sim_without_adjective)

        # Determine if adjective is hallucinated
        hallucinated = angle_with_adjective > angle_without_adjective

        # Store result for each adjective
        results.append({
            "original_caption": caption,
            "adjective": adjective,
            "adjective_free_caption": adjective_free_caption,
            "angle_with_adjective (radians)": angle_with_adjective,
            "angle_without_adjective (radians)": angle_without_adjective,
            "hallucinated": hallucinated
        })

    return results

In [5]:
# Load Hugging Face Dataset
def load_hf_dataset(dataset_name="wlsdml357/adj_image_caption_pair"):
    return load_dataset(dataset_name)

# Process Multiple Images and Captions
def process_dataset(dataset, model, processor, device, num_samples=5):
    results = []
    for idx in range(num_samples):
        # Load image and caption
        image = dataset["test"][idx]["image"]
        caption = dataset["test"][idx]["text"]

        # Extract adjectives from caption
        adjectives = extract_adjectives(caption)

        # Get image embedding once for each image
        image_embedding = get_image_embedding(model, processor, image, device)

        # Compare each adjective and check if it's hallucinated
        result = compare_adjective_embeddings(caption, adjectives, image_embedding, model, processor, device)
        results.extend(result)  # Add results for each adjective

        # Print progress
        print(f"Processed {idx+1}/{num_samples}")

    return results

# Save Results to a CSV File
def save_results_to_csv(results, output_file="results.csv"):
    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, processor = load_clip_model(device)

    # Load Hugging Face Dataset
    dataset = load_hf_dataset()

    # Process multiple samples
    results = process_dataset(dataset, model, processor, device, num_samples=2)

    # Save results to CSV
    save_results_to_csv(results)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/310 [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Processed 1/2
Processed 2/2
Results saved to results.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')